 <div class="alert alert-block alert-info"><font size="5"><center><b><u>Projecte Data Science </u></b></center></font>   
 <div class="alert alert-block alert-info"><font size="5"><center><b><u>CryptoPunks</u></b></center></font>   



\begin{align*}Anton\:Sobrino\end{align*}

\begin{align*}Juliol\hspace{2mm}\hspace{2mm}2021\end{align*}</div>

In [1]:
#Importació llibreries bàsiques 

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
import random
import re

#Importació Llibrereis per webscrapping
from selenium import webdriver

#Importacion 
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import time

#SQL
import mysql.connector
from sqlalchemy import create_engine

#warnings
import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.expand_frame_repr', False)


driver = webdriver.Chrome(executable_path=r'C:\Users\Anton SL\bin_chrome\Chromedriver.exe')
#engine = create_engine('mysql+mysqlconnector://root:Zxcvbn123+-@' + 127001 +'/cryptopunks', echo=False)
time.sleep(random.uniform(2.0,2.0))

# profile report


K = 1000
M = 1000000

### <span style=color:darkred> CriptoPunks Transacciones</span>

##### Aquesta tbl per obtenir les transacion esta sense us. vna obtindre les dades dels companys. funcionava pero calia millorar els temps de espera.

# <span style=color:darkblue>A. Obtencio dades des de la web de Crypto Punks</span>

### <span style=color:darkred>1. tbl Master</span>

##### Aquesta tbl serveix per ordenar tota la base de dades 

In [2]:
master_df = pd.DataFrame(range(0,10000), columns=['NumCriptoPunk'])
master_df.to_csv('master.csv')
master_df

,NumCriptoPunk
0,0
1,1
2,2
3,3
4,4
...,...
9995,9995
9996,9996
9997,9997
9998,9998


In [3]:
master_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 1 columns):
 #   Column         Non-Null Count  Dtype
---  ------         --------------  -----
 0   NumCriptoPunk  10000 non-null  int64
dtypes: int64(1)
memory usage: 78.2 KB


### <span style=color:darkred>2. tbl Top Sales</span>

##### Aquesta tbl obtinguda www.larvalabs.com ens infrorma del ranking de mes venuts i podem ferla servir per fer comapracions. Es pot actualitzar diariament.

In [4]:
url1 = 'https://www.larvalabs.com/cryptopunks/topsales?sortByUSD=false'
driver.get(url1)

In [5]:
MyListaAnidada=[]

for n in range(1, 100):
    try:
        CPunks = driver.find_element_by_xpath('/html/body/div[2]/div/div[2]/div['+ str(n)+ ']').text
        mylista = CPunks.split()
        MyListaAnidada.append(mylista[0:3])
    except:
        pass

df_topsales= pd.DataFrame(MyListaAnidada,columns =['Ranking','Num_CriptoPunk','Valor_ETH'])
df_topsales['Num_CriptoPunk'] = df_topsales['Num_CriptoPunk'].str.lstrip('#').astype('int64')
df_topsales['Valor_ETH']=df_topsales['Valor_ETH'].str.rstrip('Ξ') 
df_topsales['Valor_ETH']=df_topsales['Valor_ETH'].str.rstrip('K') 
df_topsales['Valor_ETH']=pd.to_numeric(df_topsales['Valor_ETH'])
df_topsales['Ranking']=pd.to_numeric(df_topsales['Ranking'])
df_topsales.to_csv('topsales.csv')
df_topsales.head(5)

,Ranking,Num_CriptoPunk,Valor_ETH
0,1,3100,4.2
1,2,7804,4.2
2,3,6965,800.0
3,4,2140,750.0
4,5,3011,667.0


In [6]:
df_topsales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60 entries, 0 to 59
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Ranking         60 non-null     int64  
 1   Num_CriptoPunk  60 non-null     int64  
 2   Valor_ETH       60 non-null     float64
dtypes: float64(1), int64(2)
memory usage: 1.5 KB


### <span style=color:darkred>3.tbl types of Crypto Punks</span>

##### tbl obtinguda de www.larvalabs.com amb la informació deks diferents types

In [7]:
url1 = 'https://www.larvalabs.com/cryptopunks/attributes'
driver.get(url1)


In [8]:
rows = len(driver.find_elements_by_xpath('/html/body/div[2]/div/div[3]/div/div[1]/table/tbody/tr'))
col = len(driver.find_elements_by_xpath('/html/body/div[2]/div/div[3]/div/div[1]/table/thead/tr[2]/th'))


print('col:', str(col), 'i rows:', str(rows) )

col: 6 i rows: 5


In [9]:
col_nom = []
for n in range(1,rows + 1):
    if n -1  < col :
        n == n-1
        col_nom.append(driver.find_element_by_xpath('/html/body/div[2]/div/div[3]/div/div[1]/table/thead/tr[2]/th[' + str(n) + ']').text)

col_nom

['Attribute', '#', 'Avail', 'Avg Sale', 'Cheapest']

In [10]:
MyListaAnidada = []
for n in range (1,rows + 1 ):
    mylista=[]
    for b in range (1, col):
        dato= driver.find_element_by_xpath('/html/body/div[2]/div/div[3]/div/div[1]/table/tbody/tr[' + str(n) + ']/td[' +str(b) +']').text
        mylista.append(dato)
        if len(mylista)== len(col_nom):
            MyListaAnidada.append(mylista)

df_types = pd.DataFrame(MyListaAnidada, columns = col_nom)
df_types.columns = ['Attribute', 'NumPunks', 'Avail', 'Avg_Sale_ETH', 'Cheapest_ETH']
#df_types['Avg_Sale_ETH'] = df_types['Avg_Sale_ETH'].str.extract(r'(0-9.\-)')
df_types['Avg_Sale_ETH']=df_types['Avg_Sale_ETH'].str.rstrip('Ξ') 
df_types['Avg_Sale_ETH']=df_types['Avg_Sale_ETH'].str.rstrip('K')
df_types['Cheapest_ETH']=df_types['Cheapest_ETH'].str.rstrip('Ξ') 
df_types['Cheapest_ETH']=df_types['Cheapest_ETH'].str.rstrip('K')
df_types['NumPunks']=pd.to_numeric(df_types['NumPunks'])
df_types['Avail']=pd.to_numeric(df_types['Avail'])
df_types['Avg_Sale_ETH']=pd.to_numeric(df_types['Avg_Sale_ETH']) 
df_types['Cheapest_ETH']=pd.to_numeric(df_types['Cheapest_ETH'])
df_types = df_types.fillna(0)
df_types.to_csv('types.csv')
df_types

,Attribute,NumPunks,Avail,Avg_Sale_ETH,Cheapest_ETH
0,Alien,9,0,0.00,0.00
1,Ape,24,4,0.00,1.60
2,Zombie,88,10,420.86,420.00
3,Female,3840,671,29.88,17.75
4,Male,6039,986,27.02,17.85


In [11]:
df_types.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Attribute     5 non-null      object 
 1   NumPunks      5 non-null      int64  
 2   Avail         5 non-null      int64  
 3   Avg_Sale_ETH  5 non-null      float64
 4   Cheapest_ETH  5 non-null      float64
dtypes: float64(2), int64(2), object(1)
memory usage: 328.0+ bytes


### <span style=color:darkred>4. tbl Skin</span>

##### tbl obtinguda de https://defypunk.com/punks/ amb la informació deks diferents types

In [12]:
url1 = 'https://defypunk.com/punks/'
driver.get(url1)

In [13]:
skin=(driver.find_element_by_xpath('//*[@id="root"]/div/main/div/div/div[2]/div[2]/button')).text

In [14]:
mylista = []
MyListaAnidada=[]
df_skin = pd.DataFrame(columns=['skin','NumP', 'Punks'])

for i in range(1,10):
    try:
        skin=(driver.find_element_by_xpath('//*[@id="root"]/div/main/div/div/div[2]/div['+ str(i) +']/button')).text
        mylista.append(skin.split())
    except:
        pass
                       
df_skin = pd.DataFrame(mylista,columns=['skin','NumP', 'Punks'])
df_skin = df_skin.drop(['Punks'],axis=1)
df_skin['NumP']=df_skin['NumP'].str.replace('(','')
df_skin['NumP']=df_skin['NumP'].str.replace(',','')
df_skin['NumP']=pd.to_numeric(df_skin['NumP'])
df_skin.to_csv('skin.csv')
df_skin

,skin,NumP
0,Alien,9
1,Ape,24
2,Zombie,88
3,Albino,1018
4,Dark,2824
5,Light,3006
6,Mid,3031


In [15]:
df_skin.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7 entries, 0 to 6
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   skin    7 non-null      object
 1   NumP    7 non-null      int64 
dtypes: int64(1), object(1)
memory usage: 240.0+ bytes


### <span style=color:darkred>5. tbl Attibutes</span>

##### tbl obtinguda de https://www.larvalabs.com/cryptopunks/attributes' amb la informació deks diferents types

In [16]:
url1 = 'https://www.larvalabs.com/cryptopunks/attributes'
driver.get(url1)

In [17]:
rows = len(driver.find_elements_by_xpath('/html/body/div[2]/div/div[3]/div/div[2]/table/tbody/tr'))
col = len(driver.find_elements_by_xpath('/html/body/div[2]/div/div[3]/div/div[2]/table/thead/tr[2]/th'))


print('col:', str(col), 'i rows:', str(rows) )

col: 6 i rows: 87


In [18]:
col_nom = []
for n in range(1,col + 1):
    if n   < col :
        n == n -1
        col_nom.append(driver.find_element_by_xpath('/html/body/div[2]/div/div[3]/div/div[2]/table/thead/tr[2]/th[' + str(n) + ']').text)

col_nom

['Attribute', '#', 'Avail', 'Avg Sale', 'Cheapest']

In [19]:
MyListaAnidada = []
for n in range (1,rows + 1 ):
    mylista=[]
    for b in range (1, col):
        dato= driver.find_element_by_xpath('/html/body/div[2]/div/div[3]/div/div[2]/table/tbody/tr[' + str(n) + ']/td[' +str(b) +']').text
        mylista.append(dato)
        if len(mylista)== len(col_nom):
            MyListaAnidada.append(mylista)
            
df_Attibutes = pd.DataFrame(MyListaAnidada, columns = col_nom)
df_Attibutes.columns = ['Attribute', 'NumPunks', 'Avail', 'Avg_Sale_ETH', 'Cheapest_ETH']
df_Attibutes['Avg_Sale_ETH']=df_Attibutes['Avg_Sale_ETH'].str.rstrip('Ξ') 
df_Attibutes['Avg_Sale_ETH']=df_Attibutes['Avg_Sale_ETH'].str.rstrip('K')
df_Attibutes['Cheapest_ETH']=df_Attibutes['Cheapest_ETH'].str.rstrip('Ξ') 
df_Attibutes['Cheapest_ETH']=df_Attibutes['Cheapest_ETH'].str.rstrip('K')
df_Attibutes['Avg_Sale_ETH']=pd.to_numeric(df_Attibutes['Avg_Sale_ETH'])
df_Attibutes['Cheapest_ETH']=pd.to_numeric(df_Attibutes['Cheapest_ETH'])
df_Attibutes['NumPunks']=pd.to_numeric(df_Attibutes['NumPunks'])
df_Attibutes['Avail']=pd.to_numeric(df_Attibutes['Avail'])
df_Attibutes = df_Attibutes.fillna(0)
df_Attibutes

,Attribute,NumPunks,Avail,Avg_Sale_ETH,Cheapest_ETH
0,Beanie,44,12,152.00,199.99
1,Choker,48,7,67.33,75.00
2,Pilot Helmet,54,12,121.20,130.00
3,Tiara,55,14,67.00,68.88
4,Orange Side,68,15,96.03,65.56
5,Buck Teeth,78,16,47.96,49.98
6,Welding Goggles,86,18,57.40,48.00
7,Pigtails,94,9,49.83,39.50
8,Pink With Hat,95,16,45.51,40.50
9,Top Hat,115,37,59.48,43.00


In [20]:
df_Attibutes = df_Attibutes.append({'Attribute': 'no accesories', 'NumPunks' : 0, 'Avail' : 0 , 'Avg_Sale_ETH': 0 , 'Cheapest_ETH' : 0 }, ignore_index=True)

In [21]:
df_Attibutes

,Attribute,NumPunks,Avail,Avg_Sale_ETH,Cheapest_ETH
0,Beanie,44,12,152.00,199.99
1,Choker,48,7,67.33,75.00
2,Pilot Helmet,54,12,121.20,130.00
3,Tiara,55,14,67.00,68.88
4,Orange Side,68,15,96.03,65.56
5,Buck Teeth,78,16,47.96,49.98
6,Welding Goggles,86,18,57.40,48.00
7,Pigtails,94,9,49.83,39.50
8,Pink With Hat,95,16,45.51,40.50
9,Top Hat,115,37,59.48,43.00


In [22]:
df_Attibutes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88 entries, 0 to 87
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Attribute     88 non-null     object 
 1   NumPunks      88 non-null     int64  
 2   Avail         88 non-null     int64  
 3   Avg_Sale_ETH  88 non-null     float64
 4   Cheapest_ETH  88 non-null     float64
dtypes: float64(2), int64(2), object(1)
memory usage: 3.6+ KB


In [23]:
df_Attibutes.to_csv('Attibutes.csv')

### <span style=color:darkred>6. tbl accesories</span>

##### Tbl Aram

In [24]:
punks_df= pd.read_csv('Punks.csv')

In [25]:
punks_df.head(5)

,Unnamed: 0,id,type,count,accesories
0,0,0,Female,3,Green Eye Shadow / Earring / Blonde Bob
1,1,1,Male,2,Smile / Mohawk
2,2,2,Female,1,Wild Hair
3,3,3,Male,3,Wild Hair / Nerd Glasses / Pipe
4,4,4,Male,4,Big Shades / Wild Hair / Earring / Goat


In [26]:
punks_df= punks_df.drop(['Unnamed: 0'], axis=1)

In [27]:
punks_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          10000 non-null  int64 
 1   type        10000 non-null  object
 2   count       10000 non-null  int64 
 3   accesories  10000 non-null  object
dtypes: int64(2), object(2)
memory usage: 312.6+ KB


In [28]:
df = pd.DataFrame(punks_df.accesories)

In [29]:
df1 = df['accesories'].str.split('/', expand=True).stack()

In [30]:
df1

0     0     Green Eye Shadow 
      1              Earring 
      2            Blonde Bob
1     0                Smile 
      1                Mohawk
                  ...        
9998  0      Wild White Hair 
      1       Black Lipstick 
      2      Clown Eyes Green
9999  0               Mohawk 
      1          Nerd Glasses
Length: 27547, dtype: object

In [31]:
df1 = df1.reset_index(level=1,drop=True).rename('accesories1')

In [32]:
df1

0        Green Eye Shadow 
0                 Earring 
0               Blonde Bob
1                   Smile 
1                   Mohawk
               ...        
9998      Wild White Hair 
9998       Black Lipstick 
9998      Clown Eyes Green
9999               Mohawk 
9999          Nerd Glasses
Name: accesories1, Length: 27547, dtype: object

In [33]:
punks_df=punks_df.drop('accesories', axis=1)


In [34]:
punks_df = punks_df.join(df1)

In [35]:
punks_df['count'] = 1

In [36]:
punks_df

,id,type,count,accesories1
0,0,Female,1,Green Eye Shadow
0,0,Female,1,Earring
0,0,Female,1,Blonde Bob
1,1,Male,1,Smile
1,1,Male,1,Mohawk
...,...,...,...,...
9998,9998,Female,1,Wild White Hair
9998,9998,Female,1,Black Lipstick
9998,9998,Female,1,Clown Eyes Green
9999,9999,Female,1,Mohawk


In [37]:
punks_df = punks_df.reset_index(drop=True)

In [38]:
punks_df["type"] = punks_df["type"].str.extract(r'(\w+)')
punks_df["accesories1"] = punks_df["accesories1"].str.extract(r'(\w+\D+)')
punks_df["accesories2"] = punks_df["accesories1"].str.rstrip()

punks_df
#df_Attibutes['Attribute'] = df_Attibutes['Attribute'].str.lower() 

,id,type,count,accesories1,accesories2
0,0,Female,1,Green Eye Shadow,Green Eye Shadow
1,0,Female,1,Earring,Earring
2,0,Female,1,Blonde Bob,Blonde Bob
3,1,Male,1,Smile,Smile
4,1,Male,1,Mohawk,Mohawk
...,...,...,...,...,...
27542,9998,Female,1,Wild White Hair,Wild White Hair
27543,9998,Female,1,Black Lipstick,Black Lipstick
27544,9998,Female,1,Clown Eyes Green,Clown Eyes Green
27545,9999,Female,1,Mohawk,Mohawk


In [39]:
punks_df['idaccesories']=''

In [40]:
for i in range(len(punks_df)):
    myacc =punks_df['accesories2'].iloc[i]
    for n in range(len(df_Attibutes)):
        if myacc == df_Attibutes.Attribute[n]:
            punks_df['idaccesories'][i] = n


In [42]:
punks_df.isnull().sum()

id              0
type            0
count           0
accesories1     8
accesories2     8
idaccesories    0
dtype: int64

In [43]:
punks_df['idaccesories'] = punks_df['idaccesories'].apply(lambda x : 87 if x == '' else x)

In [44]:
#no tenen atribust
punks_df = punks_df.drop('accesories1', axis=1)

In [45]:

punks_df.isnull().sum()

id              0
type            0
count           0
accesories2     8
idaccesories    0
dtype: int64

In [47]:
punks_df.to_csv('punks_accesories.csv')

### <span style=color:darkred>7. tbl historico</span>

##### Tbl Aram

In [48]:
hist_df = pd.read_csv('transaccions_hist_df.csv')
hist_df

,Unnamed: 0,cryptopunk_number,txn_type,from,to,txn_date,amount_dolars,amount_eth,dolar_factors,eth_factors
0,1,0,Bid Withdrawn,0x983ace,NaN,2021-05-10,1.03,250.00,1000000.0,NaN
1,2,0,Bid,0x983ace,NaN,2021-04-11,537615.00,250.00,NaN,NaN
2,3,0,Bid Withdrawn,0xd7510a,NaN,2021-03-03,160973.00,100.00,NaN,NaN
3,4,0,Bid,0xd7510a,NaN,2021-02-20,188897.00,100.00,NaN,NaN
4,5,0,Bid Withdrawn,natealex,NaN,2020-09-16,25532.00,692.00,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
123269,133269,9999,Bid,0xa0a59c,NaN,2017-07-03,69.00,0.25,NaN,NaN
123270,133270,9999,Bid,0x717403,NaN,2017-06-26,51.00,0.20,NaN,NaN
123271,133271,9999,Bid,ddaavvee,NaN,2017-06-23,65.00,0.20,NaN,NaN
123272,133272,9999,Bid,0x5b098b,NaN,2017-06-23,50.00,0.15,NaN,NaN


In [49]:
hist_df.isnull().sum().sum()

453248

In [50]:
hist_df.to = hist_df.to.fillna('')


In [51]:
hist_df['from'] = hist_df['from'].fillna('')

In [52]:
hist_df [['amount_dolars','amount_eth','dolar_factors','eth_factors']] = hist_df[['amount_dolars','amount_eth','dolar_factors','eth_factors']].fillna(0)

In [53]:
hist_df.isnull().sum().sum()

0

In [54]:
hist['txn_date'] =to_datetime(hist['txn_date'])

In [55]:
hist_df= hist_df.drop('Unnamed: 0',axis=1)

In [56]:
hist_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 123274 entries, 0 to 123273
Data columns (total 9 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   cryptopunk_number  123274 non-null  int64  
 1   txn_type           123274 non-null  object 
 2   from               123274 non-null  object 
 3   to                 123274 non-null  object 
 4   txn_date           123274 non-null  object 
 5   amount_dolars      123274 non-null  float64
 6   amount_eth         123274 non-null  float64
 7   dolar_factors      123274 non-null  float64
 8   eth_factors        123274 non-null  float64
dtypes: float64(4), int64(1), object(4)
memory usage: 8.5+ MB


In [57]:
hist_df.to_csv('hist_df.csv')

# <span style=color:darkblue>B. Creació BBDD MySql</span>

### <span style=color:darkred>Scripts MySQL </span>

### <span style=color:darkred>Scripts MySQL </span>

In [38]:
mydb = mysql.connector.connect(
  host="127.0.0.1",
  user="root",
  password="Zxcvbn123+-",
  database='cryptopunks'
)

print(mydb)

In [39]:
mycursor = mydb.cursor()

In [40]:
#df_topsales.to_sql(name='topsales', con=mydb, if_exists = 'append', index=False)